In [ ]:
from argparse import ArgumentParser
from datetime import datetime, timedelta
from pathlib import Path
from pprint import pprint as pp
import requests

***DON'T USE API KEY***

We have limited credits. Alejandro is going to contact purpleair as they said something about being able to get data for free.

We can use this for some initial testing, but don't want to set up anything regular *yet*.

In [ ]:
# API_KEY = "111D90AF-EC2C-11EE-B9F7-42010A80000D"

# Math on sensor data volume

Trying to estimate how many 'API Points' we'd use.

An API call is 1 credit, and I *think* each field requested is 2 points

In [ ]:
n_fields = 5 #temperature, humidity, pm2.5, time_stamp, voc
average = 60 # 10 if possible
n_sensors = 100

n_samples = (24 * 60) / average # day in minutes, divided by averaging period
perday = n_samples * n_fields * n_sensors + 1
print(f"Per day:\n\t{n_samples =: }\n\t* {n_fields = :}\n\t* {n_sensors = :}\n= {int(perday):_} API credits per day")
print(f"= {int(perday * 30):_} per month")
print(f"= {int(perday * 30 * 12):_} per year")

In [ ]:
fields = [
    # station information and status fields
    "name",
    "model",
    "latitude",
    "longitude",
    "altitude",
    "uptime",
    # environmental
    "humidity",
    "temperature",
    "pressure",
    # miscellaneous
    "voc",
    # "ozone1",
    "time_stamp",
    # pm
    "pm1.0",
    "pm2.5",
    "pm10.0",
    # particle count
    "0.3_um_count",
    "0.5_um_count",
    "1.0_um_count",
    "2.5_um_count",
    "5.0_um_count",
    "10.0_um_count",
]

In [ ]:
fields = [
    # station information and status fields
    "name",
    "uptime",
]

outdir = Path("./purpleair").resolve()
outdir.mkdir(exist_ok=True)

In [ ]:
# build URL
end = datetime.today()
start = end - timedelta(days=1)
sensor_index = "OGLASG16"
fieldstr = ",".join(fields)

# averaging (10 min is default)
average = 10 # or 0, 30, 60, 360, 1440 (minutes)

params = {
    "start_timestamp": start.strftime("%F"),
    "end_timestamp": end.strftime("%F"),
    "fields": ",".join(fields),
    "average": average
}

url = f"https://api.purpleair.com/v1/sensors/"

# resp = requests.get(url, params = params, headers={"X-API-Key": API_KEY})

# The lat long box below is just a _VERY ROUGH_ approximate box around glasgow
# so that we can filter out the glasgow sensors from the global list of
# purpleair sensors
resp = requests.get(url, params = {
    'fields': 'name',
    'nwlng': -4.2827620716106525,
    'nwlat': 55.873550361651326,
    'selng': -4.228095861367214,
    'selat': 55.841992388711006,
}, headers={"X-API-Key": API_KEY})
print(resp)
pp(resp.json())
print(resp.url)


# Get a sample of data from a single glasgow sensor

In [ ]:
sensor_index = "200201"

url = f"https://api.purpleair.com/v1/sensors/{sensor_index}/history"

# resp = requests.get(url, params = params, headers={"X-API-Key": API_KEY})
resp = requests.get(
    url, params={"fields": "temperature"}, headers={"X-API-Key": API_KEY}
)
print(resp)
pp(resp.json())
print(resp.url)